## Loading the environment

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# MODEL = "gpt-3.5-turbo"
MODEL = "llama3.1"

# Sam Altman: OpenAI CEO on GPT-4, ChatGPT, and the Future of AI
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=L_Guz73e6fw"


## Setting up llm/slm model

In [4]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key=OPENAI_API_KEY,model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

Simple model test to see if it's working.

Extracting the response by chaining the model with an output parser

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser
chain.invoke("Tell me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nHope that made you laugh! Do you want to hear another one?"

## Setting up the prompt templates